# Notebook to prep functions for report

* Grouped down to the DISTRICT and the CYCLE level

In [1]:
import intake
import numpy as np
import pandas as pd
from calitp import to_snakecase
from dla_utils import _dla_utils
from IPython.display import HTML, Markdown
from siuba import *
from shared_utils import geography_utils

import altair as alt

import _data_cleaning
import _report_utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/atp/'


In [3]:
df_all = _report_utils.read_in_joined_data()

/home/jovyan/data-analyses/dla/atp/_report_utils.py:67: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)


In [4]:
pd.set_option("display.max_columns",500)

In [5]:
## paramaters cell
district = 4

In [6]:
## parameters cell
cycle = 5

In [7]:
## subset df to just the district
df = df_all>>filter(_.a2_ct_dist==district)

In [8]:
df = df>>filter(_.project_cycle==cycle)

In [9]:
# #check where everything is mapped
# df = df_all.copy()

In [10]:
display(HTML("<h2>Quick Stats</h2>"))

display(HTML(f"Out of {len(df)} Active Transportation Program Project Applications, "
            f"there are <strong>{(df>>filter(_.awarded=='Y')).project_app_id.nunique()} "
            f"projects</strong> that recieved funding over "
            f"{df.project_cycle.nunique()} cycles"))


In [11]:
df>>group_by(_.project_cycle)>>count(_.data_origin)

,project_cycle,data_origin,n
0,5,Application,60
1,5,Funded,5


In [50]:
quick_view = df>>filter(_.awarded=="Y")>>select(_.data_origin, _.a1_imp_agcy_name, _.a2_info_proj_name, 
                                   _.a2_county, _.total_project_cost)

In [51]:
quick_view['total_project_cost'] = quick_view['total_project_cost'].map('$ {:0,.2f}'.format)

In [52]:
display(HTML(_dla_utils.pretify_tables(quick_view)))

Data Origin,A1 Imp Agcy Name,A2 Info Proj Name,A2 County,Total Project Cost
Funded,"Oakland, City of",7th Street Connection Project,Alameda,"$ 21,037,000.00"
Funded,Santa Clara County,Active and Safe Routes to a Healthier City,Santa Clara,"$ 2,510,000.00"
Funded,"Oakland, City of",East Oakland Neighborhood Bike Routes,Alameda,"$ 21,859,000.00"
Funded,Contra Costa County,North Bailey Road Active Transportation Corridor,Contra Costa,"$ 6,845,000.00"
Funded,"Fairfield, City of",West Texas Street Complete Streets Project,Solano,"$ 16,922,000.00"


In [53]:
df>>group_by(_.awarded)>>count(_.a2_county)>>arrange(_.a2_county)

,awarded,a2_county,n
0,N,Alameda,17
9,Y,Alameda,2
1,N,Contra Costa,16
10,Y,Contra Costa,1
2,N,Marin,5
11,Y,Marin,0
3,N,Napa,3
12,Y,Napa,0
4,N,San Francisco,2
13,Y,San Francisco,0


## mapping

In [55]:
df.sample()

,project_app_id,project_cycle,a1_locode,#,atp_id,awarded,ppno,ppno_1,data_origin,geometry,project_status,solicitation_abv,solicitation,soliciting_agency,project_size,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,a1_proj_partner_exists,assembly_district,congressional_district,senate_district,a2_county,a2_ct_dist,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a2_rtpa,a2_proj_scope_summary,a1_imp_agcy_fed_ma_num,a1_imp_agcy_ma,a1_imp_agcy_state_ma_num,a1_imp_agcy_street,a1_imp_agcy_title,a1_imp_agcy_zip,a1_letter_of_intent,a1_proj_partner_title,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_project_location_map,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_proj_type,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_trails,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,a3_current_plan,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,app_fk,details_datetime_stamp,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_

In [56]:
df_map= (df >>select(_.awarded, _.project_app_id, _.project_cycle, _.data_origin, _.geometry,
               _.a1_imp_agcy_city, _.a1_imp_agcy_name, _.a1_proj_partner_agcy, 
               _.assembly_district, _.congressional_district, _.senate_district,
              _.a2_county, _.a2_info_proj_descr, _.a2_info_proj_loc, _.a2_info_proj_name,
               _.a2_mop_uza_population, _.a2_mpo, _.a1_imp_agcy_street, _.a3_proj_type, 
               _.a3_proj_type, _['total_atp_$'], _.a2_proj_lat, _.a2_proj_long))

In [57]:
df_map = df_map>>filter(_.project_cycle==5)

In [58]:
df_map>>filter(_.geometry.isnull())

,awarded,project_app_id,project_cycle,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long
10,Y,"4-Oakland, City of-1",5.00,Funded,NaN,Oakland,"Oakland, City of",NaN,18,13,9,Alameda,Complete Streets corridor project including pr...,7th Street from Mandela Parkway to Martin Luth...,7th Street Connection Project,Project is located within one of the ten large...,MTC,250 Frank Ogawa Plaza,NaN,"14,180,000.00",37.80,-122.29
14,Y,4-Santa Clara County-1,5.00,Funded,NaN,San Jose,Santa Clara County,NaN,30,"19, 20",17,Santa Clara,Non-infrastructure 6 E’s project linking key c...,"Location is the City of Gilroy, particularly f...",Active and Safe Routes to a Healthier City,Project is located within one of the ten large...,MTC,70 W Hedding,NaN,"2,510,000.00",37.01,121.57
32,Y,"4-Oakland, City of-2",5.00,Funded,NaN,Oakland,"Oakland, City of",NaN,18,13,9,Alameda,Neighborhood bike routes on four corridors in ...,"Neighborhood bike routes on 81st Avenue, 85th ...",East Oakland Neighborhood Bike Routes,Project is located within one of the ten large...,MTC,250 Frank H Ogawa Plaza,NaN,"17,269,000.00",37.75,-122.18
29,Y,4-Contra Costa County-2,5.00,Funded,NaN,Martinez,Contra Costa County,NaN,14,11,7,Contra Costa,"Construct two-way cycle track, ADA-compliant c...","Bailey Road in unincorporated Bay Point, bound...",North Bailey Road Active Transportation Corridor,Project is located within one of the ten large...,MTC,255 Glacier Drive,NaN,"6,159,000.00",38.02,121.94
2,Y,"4-Fairfield, City of-1",5.00,Funded,NaN,Fairfield,"Fairfield, City of",NaN,11,3,3,Solano,Construct Class IV separated bikeways with Cla...,West Texas Street between Beck Avenue and Penn...,West Texas Street Complete Streets Project,Project is located within one of the ten large...,MTC,1000 Webster Street,NaN,"10,903,000.00",38.25,-122.06


In [59]:
## reapply geometry col for lat long
df_map = (geography_utils.create_point_geometry(df_map, longitude_col = 'a2_proj_long', latitude_col = 'a2_proj_lat'))

In [60]:
df_map

,awarded,project_app_id,project_cycle,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long
154,N,"4-San Mateo, City of-1",5,Application,POINT (-122.30526 37.55207),San Mateo,"San Mateo, City of",NaN,22,14,13,San Mateo,"PS&E, ROW, CON funding for design/construction...",The project extents are South Delaware Street ...,Delaware Street Safe Routes to School Corridor,Project is located within one of the ten large...,MTC,330 W. 20th Avenue,Infrastructure - Small,NaN,37.55,-122.31
443,N,"4-San Rafael, City of-1",5,Application,POINT (-122.51269 37.96830),San Rafael,"San Rafael, City of",NaN,10,2,2,Marin,Ped/bike crossing of San Rafael Canal between ...,Crossing between Canal Street in San Rafael’s ...,San Rafael Canal Crossing Project,Project is located within one of the ten large...,MTC,111 Morphew Street,Infrastructure - Large,NaN,37.97,-122.51
255,N,"4-San Ramon, City of-1",5,Application,POINT (-121.96690 37.78049),San Ramon,"San Ramon, City of",NaN,16,15,7,Contra Costa,RTP ID: 17-02-0003 \rBicycle and Pedest...,This project is located in the City of San Ram...,Iron Horse Trail Bicycle and Pedestrian Overcr...,Project is located within one of the ten large...,MTC,7000 Bollinger Canyon Road,Infrastructure - Large,NaN,37.78,-121.97
372,N,"4-Vallejo, City of-1",5,Application,POINT (-122.23000 38.07000),Vallejo,"Vallejo, City of",NaN,Needs Manual Assistance,5,3,Solano,A recreational trail of approximately 1.97 mil...,From intersection of Sequoia Avenue and Lincol...,Vallejo Bluff Trail Phase 1,Project is located within one of the ten large...,MTC,555 Santa Clara St,Infrastructure - Medium,NaN,38.07,-122.23
341,N,"4-Santa Rosa, City of-1",5,Application,POINT (-122.72602 38.45589),SANTA ROSA,"Santa Rosa, City of",NaN,2,5,2,Sonoma,Construct a Class I ADA accessible and mode-se...,"In the City of Santa Rosa, Sonoma County, over...",Santa Rosa US Highway 101 Bicycle and Pedestri...,Project is located within one of the ten large...,MTC,69 STONY CIRCLE,Infrastructure - Large,NaN,38.46,-122.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Y,"4-Oakland, City of-1",5.00,Funded,POINT (-122.28668 37.80356),Oakland,"Oakland, City of",NaN,18,13,9,Alameda,Complete Streets corridor project including pr...,7th Street from Mandela Parkway to Martin Luth...,7th Street Connection Project,Project is located within one of the ten large...,MTC,250 Frank Ogawa Plaza,NaN,"14,180,000.00",37.80,-122.29
14,Y,4-Santa Clara County-1,5.00,Funded,POINT (121.56830 37.00580),San Jose,Santa Clara County,NaN,30,"19, 20",17,Santa Clara,Non-infrastructure 6 E’s project linking key c...,"Location is the City of Gilroy, particularly f...",Active and Safe Routes to a Healthier City,Project is located within one of the ten large...,MTC,70 W Hedding,NaN,"2,510,000.00",37.01,121.57
32,Y,"4-Oakland, City of-2",5.00,Funded,POINT (-122.18217 37.75100),Oakland,"Oakland, City of",NaN,18,13,9,Alameda,Neighborhood bike routes on four corridors in ...,"Neighborhood bike routes on 81st Avenue, 85th ...",East Oakland Neighborhood Bike Routes,Project is located within one of the ten large...,MTC,250 Frank H Ogawa Plaza,NaN,"17,269,000.00",37.75,-122.18
29,Y,4-Contra Costa County-2,5.00,Funded,POINT (121.94196 38.02416),Martinez,Contra Costa County,NaN,14,11,7,Contra Costa,"Construct two-way cycle track, ADA-compliant c...","Bailey Road in unincorporated Bay Point, bound...",North Bailey Road Active Transportation Corridor,Project is located within one of the ten large...,MTC,255 Glacier Drive,NaN,"6,159,000.00",38.02,121.94


### ~~ISSUE~~ -- SOLVED WITH A QUICK FIX
* funded data lost mapping data from geometry column

In [61]:
df_map.explore("a2_county", cmap="Blues")  

In [62]:
# issue getting some lat longs in wrong country

In [63]:
df_map>>arrange(_.a2_proj_long)

,awarded,project_app_id,project_cycle,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long
341,N,"4-Santa Rosa, City of-1",5,Application,POINT (-122.72602 38.45589),SANTA ROSA,"Santa Rosa, City of",NaN,2,5,2,Sonoma,Construct a Class I ADA accessible and mode-se...,"In the City of Santa Rosa, Sonoma County, over...",Santa Rosa US Highway 101 Bicycle and Pedestri...,Project is located within one of the ten large...,MTC,69 STONY CIRCLE,Infrastructure - Large,NaN,38.46,-122.73
304,N,4-Sonoma County-1,5,Application,POINT (-122.71861 38.39446),Santa Rosa,Sonoma County,NaN,1,5,2,Sonoma,"Construct sidewalks, construct trail segment, ...","Moorland Ave. (Todd Rd. to Bellevue Ave.), W. ...",Moorland Pedestrian and School Access,Project is located within one of the ten large...,MTC,"2300 County Center Drive, Suite B100",Infrastructure + NI - Medium,NaN,38.39,-122.72
101,N,"4-Novato, City of-1",5,Application,POINT (-122.60974 38.11794),Novato,"Novato, City of",NaN,10,2,2,Marin,Safe Routes to School Project that will result...,The project is located at the intersection of ...,San Marin High School Area Multimodal Access P...,Project is located within one of the ten large...,MTC,922 Machin Avenue,Infrastructure - Small,NaN,38.12,-122.61
201,N,"4-Corte Madera, Town of-1",5,Application,POINT (-122.51722 37.93656),Corte Madera,"Corte Madera, Town of",NaN,10,2,2,Marin,Funding for construction of Class I pathway on...,"Corte Madera, CA (with one intersection in Lar...",Central Marin Regional Pathways Gap Closure Pr...,Project is located within one of the ten large...,MTC,300 Tamalpais Drive,Infrastructure - Small,NaN,37.94,-122.52
443,N,"4-San Rafael, City of-1",5,Application,POINT (-122.51269 37.96830),San Rafael,"San Rafael, City of",NaN,10,2,2,Marin,Ped/bike crossing of San Rafael Canal between ...,Crossing between Canal Street in San Rafael’s ...,San Rafael Canal Crossing Project,Project is located within one of the ten large...,MTC,111 Morphew Street,Infrastructure - Large,NaN,37.97,-122.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,N,4-California Department of Transportation-3,5,Application,POINT (122.31089 37.90070),Oakland,California Department of Transportation,NaN,15,11,9,Contra Costa,Improve pedestrian/bicycle access at Central A...,The Project is located on Central Avenue cross...,Central Avenue I-80 Undercrossing Ped/Bike Imp...,Project is located within one of the ten large...,MTC,District 4 Office - 111 Grand Avenue,Infrastructure - Medium,NaN,37.90,122.31
328,N,4-Contra Costa County-3,5,Application,POINT (122.36330 37.95871),Martinez,Contra Costa County,NaN,15,11,9,Contra Costa,Construct widened and ADA-accessible sidewalks...,On Market Avenue between Fred Jackson Way and ...,Market Avenue Complete Street,Project is located within one of the ten large...,MTC,255 Glacier Drive,Infrastructure - Medium,NaN,37.96,122.36
185,N,4-Napa County-1,5,Application,POINT (122.39940 38.43341),Napa,Napa County,NaN,4,5,3,Napa,The project consists of an 8.06 mile ADA compl...,The project is located in the unincorporated a...,Napa Valley Vine Trail Yountville to St Helena...,Project is located within one of the ten large...,MTC,1195 Third Street Suite 101,Infrastructure - Large,NaN,38.43,122.40
312,N,"4-Tiburon, Town of-1",5,Application,POINT (122.48793 37.89808),Tiburon,"Tiburon, Town of",NaN,10,2,2,Marin,Trestle Glen Boulevard Class II Bikeway,Trestle Glen Boulevard between Tiburon Bouleva...,Trestle Glen Boulevard Class II Bikeway,Project is located within one of the ten large...,MTC,1505 Tiburon Boulevard,Infrastructure - Medium,NaN,37.90,122.49


###  ~~ISSUE: Some Project Longs are wrong~~ - SOLVED WITH A QUICK FIX

In [64]:
df_map.a2_proj_long.info()


<class 'pandas.core.series.Series'>
Int64Index: 65 entries, 154 to 2
Series name: a2_proj_long
Non-Null Count  Dtype  
--------------  -----  
65 non-null     float64
dtypes: float64(1)
memory usage: 1.0 KB


In [65]:
df_map.sample()

,awarded,project_app_id,project_cycle,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long
32,Y,"4-Oakland, City of-2",5.00,Funded,POINT (-122.18217 37.75100),Oakland,"Oakland, City of",NaN,18,13,9,Alameda,Neighborhood bike routes on four corridors in ...,"Neighborhood bike routes on 81st Avenue, 85th ...",East Oakland Neighborhood Bike Routes,Project is located within one of the ten large...,MTC,250 Frank H Ogawa Plaza,NaN,"17,269,000.00",37.75,-122.18


In [66]:
df_map_correct = (df_map>>filter(_.a2_proj_long<0))

In [67]:
df_map_incorrect = (df_map>>filter(_.a2_proj_long>0))

In [68]:
df_map_incorrect['a2_proj_long'] = df_map_incorrect['a2_proj_long'] * (-1)

/opt/conda/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [69]:
df_map_corrected = (geography_utils.create_point_geometry(df_map_incorrect, longitude_col = 'a2_proj_long', latitude_col = 'a2_proj_lat'))

In [70]:
df_map2 = (pd.concat([df_map_corrected, df_map_correct]))

In [71]:
df_map2

,awarded,project_app_id,project_cycle,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long
399,N,4-Solano County-1,5,Application,POINT (-122.23833 38.09639),Fairfield,Solano County,NaN,14,5,3,Solano,"Remove and replace deficient sidewalk, curb, g...",On Benicia Road between Beach Street and Lemon...,Benicia Road Complete Street Project,Project is located within one of the ten large...,MTC,"675 Texas Street, Suite 5500",Infrastructure - Medium,NaN,38.10,-122.24
312,N,"4-Tiburon, Town of-1",5,Application,POINT (-122.48793 37.89808),Tiburon,"Tiburon, Town of",NaN,10,2,2,Marin,Trestle Glen Boulevard Class II Bikeway,Trestle Glen Boulevard between Tiburon Bouleva...,Trestle Glen Boulevard Class II Bikeway,Project is located within one of the ten large...,MTC,1505 Tiburon Boulevard,Infrastructure - Medium,NaN,37.90,-122.49
434,N,"4-Vacaville, City of-1",5,Application,POINT (-121.58191 38.21294),Vacaville,"Vacaville, City of",NaN,11,3,3,Solano,"Phase 1 of Class 1 gap closure, path improveme...","Class I Path from Allison Dr, along both sides...",Ulatis Creek Safe Routes to Transit Gap Closure,Project is located within one of the ten large...,MTC,"650 Merchant St, Vacaville, CA 95688",Infrastructure - Medium,NaN,38.21,-121.58
367,N,"4-San Jose, City of-5",5,Application,POINT (-121.51000 37.20000),San Jose,"San Jose, City of",NaN,"27, 25",19,"15, 10",Santa Clara,Development of 1.95 mile trail (Class I Bikeway),"Project corridor begins at Story Rd, follows t...",Five Wounds Trail (Story to Julian) - PA&ED an...,Project is located within one of the ten large...,MTC,200 East Santa Clara Street,Infrastructure - Large,NaN,37.20,-121.51
328,N,4-Contra Costa County-3,5,Application,POINT (-122.36330 37.95871),Martinez,Contra Costa County,NaN,15,11,9,Contra Costa,Construct widened and ADA-accessible sidewalks...,On Market Avenue between Fred Jackson Way and ...,Market Avenue Complete Street,Project is located within one of the ten large...,MTC,255 Glacier Drive,Infrastructure - Medium,NaN,37.96,-122.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,N,4-Metropilitan Transportation Commission (MTC)-1,5,Application,POINT (-122.48389 37.94532),San Francisco,Metropilitan Transportation Commission (MTC),NaN,10,2,2,Marin,Construction of a multiuse pathway to connect ...,"Francisco Boulevard East in San Rafael, from t...",Richmond-San Rafael Bridge Shared Use Path Gap...,Project is located within one of the ten large...,MTC,375 Beale Street; Suite 800,Infrastructure - Medium,NaN,37.95,-122.48
330,N,"4-Millbrae, City of-1",5,Application,POINT (-122.38083 37.60239),Millbrae,"Millbrae, City of",NaN,22,14,13,San Mateo,Across US 101 north of and adjacent to the exi...,The project is located adjacent to Millbrae Av...,Millbrae Avenue Bicycle/Pedestrian Overcrossin...,Project is located within one of the ten large...,MTC,621 Magnolia Avenue,Infrastructure - Large,NaN,37.60,-122.38
10,Y,"4-Oakland, City of-1",5.00,Funded,POINT (-122.28668 37.80356),Oakland,"Oakland, City of",NaN,18,13,9,Alameda,Complete Streets corridor project including pr...,7th Street from Mandela Parkway to Martin Luth...,7th Street Connection Project,Project is located within one of the ten large...,MTC,250 Frank Ogawa Plaza,NaN,"14,180,000.00",37.80,-122.29
32,Y,"4-Oakland, City of-2",5.00,Funded,POINT (-122.18217 37.75100),Oakland,"Oakland, City of",NaN,18,13,9,Alameda,Neighborhood bike routes on four corridors in ...,"Neighborhood bike routes on 81st Avenue, 85th ...",East Oakland Neighborhood Bike Routes,Project is located within one of the ten large...,MTC,250 Frank H Ogawa Plaza,NaN,"17,269,000.00",37.75,-122.18


In [72]:
#check 
df_map2.explore("project_cycle", cmap="Blues")  

/opt/conda/lib/python3.10/site-packages/geopandas/explore.py:412: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)


In [73]:
df_map2>>arrange(_.a2_proj_long)

,awarded,project_app_id,project_cycle,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long
292,N,"4-Healdsburg, City of-1",5,Application,POINT (-122.87310 38.63250),Healdsburg,"Healdsburg, City of",NaN,02,02,02,Sonoma,"PA&ED, ROW, PS&E, and CON funding to implement...",Healdsburg Avenue between Powell Avenue and th...,Healdsburg Avenue Complete Streets Project,Project is located within one of the ten large...,MTC,401 Grove Street,Infrastructure - Large,NaN,38.63,-122.87
341,N,"4-Santa Rosa, City of-1",5,Application,POINT (-122.72602 38.45589),SANTA ROSA,"Santa Rosa, City of",NaN,2,5,2,Sonoma,Construct a Class I ADA accessible and mode-se...,"In the City of Santa Rosa, Sonoma County, over...",Santa Rosa US Highway 101 Bicycle and Pedestri...,Project is located within one of the ten large...,MTC,69 STONY CIRCLE,Infrastructure - Large,NaN,38.46,-122.73
304,N,4-Sonoma County-1,5,Application,POINT (-122.71861 38.39446),Santa Rosa,Sonoma County,NaN,1,5,2,Sonoma,"Construct sidewalks, construct trail segment, ...","Moorland Ave. (Todd Rd. to Bellevue Ave.), W. ...",Moorland Pedestrian and School Access,Project is located within one of the ten large...,MTC,"2300 County Center Drive, Suite B100",Infrastructure + NI - Medium,NaN,38.39,-122.72
101,N,"4-Novato, City of-1",5,Application,POINT (-122.60974 38.11794),Novato,"Novato, City of",NaN,10,2,2,Marin,Safe Routes to School Project that will result...,The project is located at the intersection of ...,San Marin High School Area Multimodal Access P...,Project is located within one of the ten large...,MTC,922 Machin Avenue,Infrastructure - Small,NaN,38.12,-122.61
201,N,"4-Corte Madera, Town of-1",5,Application,POINT (-122.51722 37.93656),Corte Madera,"Corte Madera, Town of",NaN,10,2,2,Marin,Funding for construction of Class I pathway on...,"Corte Madera, CA (with one intersection in Lar...",Central Marin Regional Pathways Gap Closure Pr...,Project is located within one of the ten large...,MTC,300 Tamalpais Drive,Infrastructure - Small,NaN,37.94,-122.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,N,"4-Dublin, City of-1",5,Application,POINT (-121.91010 37.71590),Dublin,"Dublin, City of",NaN,16,15,70,Alameda,Pedestrian safety improvements near 12 schools...,City of Dublin - Installation of RRFB and bulb...,City of Dublin Safe Routes to Schools Project,Project is located within one of the ten large...,MTC,100 Civic Plaza,Infrastructure + NI - Medium,NaN,37.72,-121.91
223,N,4-Contra Costa Transportation Authority (CCTA)-1,5,Application,POINT (-121.88154 38.01909),Walnut Creek,Contra Costa Transportation Authority (CCTA),NaN,"11, 14",11,7,Contra Costa,Increase bicycling and walking trips to/from 6...,Program will serve all public K-12 schools in ...,Our Streets: SRTS Community Bike/Walk Campaign...,Project is located within one of the ten large...,MTC,"Planning Department, 2999 Oak Road, Suite 100",Non-Infrastructure,NaN,38.02,-121.88
434,N,"4-Vacaville, City of-1",5,Application,POINT (-121.58191 38.21294),Vacaville,"Vacaville, City of",NaN,11,3,3,Solano,"Phase 1 of Class 1 gap closure, path improveme...","Class I Path from Allison Dr, along both sides...",Ulatis Creek Safe Routes to Transit Gap Closure,Project is located within one of the ten large...,MTC,"650 Merchant St, Vacaville, CA 95688",Infrastructure - Medium,NaN,38.21,-121.58
14,Y,4-Santa Clara County-1,5.00,Funded,POINT (-121.56830 37.00580),San Jose,Santa Clara County,NaN,30,"19, 20",17,Santa Clara,Non-infrastructure 6 E’s project linking key c...,"Location is the City of Gilroy, particularly f...",Active and Safe Routes to a Healthier City,Project is located within one of the ten large...,MTC,70 W Hedding,NaN,"2,510,000.00",37.01,-121.57


In [74]:
df_map2>>filter(_.a2_proj_lat>300)

,awarded,project_app_id,project_cycle,data_origin,geometry,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,assembly_district,congressional_district,senate_district,a2_county,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a1_imp_agcy_street,a3_proj_type,total_atp_$,a2_proj_lat,a2_proj_long


In [75]:
df_map3 = df_map2>>filter(_.a2_proj_lat<300)

In [76]:
#check again 
df_map3.explore("project_cycle", cmap="Blues")  

/opt/conda/lib/python3.10/site-packages/geopandas/explore.py:412: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)


In [77]:
# still have some weird locations but better than before

In [78]:
## going back to just district

## Metrics

In [79]:
df.sample()

,project_app_id,project_cycle,a1_locode,#,atp_id,awarded,ppno,ppno_1,data_origin,geometry,project_status,solicitation_abv,solicitation,soliciting_agency,project_size,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,a1_proj_partner_exists,assembly_district,congressional_district,senate_district,a2_county,a2_ct_dist,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a2_rtpa,a2_proj_scope_summary,a1_imp_agcy_fed_ma_num,a1_imp_agcy_ma,a1_imp_agcy_state_ma_num,a1_imp_agcy_street,a1_imp_agcy_title,a1_imp_agcy_zip,a1_letter_of_intent,a1_proj_partner_title,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_project_location_map,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_proj_type,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_trails,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,a3_current_plan,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,app_fk,details_datetime_stamp,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_

In [80]:
unique_agencies = df>>group_by(_.project_cycle, _.a2_county, _.data_origin)>>summarize(n_unique_agency= _.a1_imp_agcy_name.nunique())

In [81]:
unique_agencies =(spread(unique_agencies, "data_origin", "n_unique_agency"))

In [82]:
unique_agencies = unique_agencies.rename(columns={"a2_county":"county_name"})

In [83]:
unique_agencies['Application'].fillna(0, inplace=True)
unique_agencies['Funded'].fillna(0, inplace=True)

In [84]:
unique_agencies['Application'] = unique_agencies['Application'].astype('int32')
unique_agencies['Funded'] = unique_agencies['Funded'].astype('int32')

In [85]:
display(HTML("<h3>Number of Unique Agencies By County</h3>"))
display(HTML(_dla_utils.pretify_tables(unique_agencies)))


Project Cycle,County Name,Application,Funded
5,Alameda,8,1
5,Contra Costa,10,1
5,Marin,5,0
5,Napa,3,0
5,San Francisco,2,0
5,San Mateo,10,0
5,Santa Clara,1,1
5,Solano,3,1
5,Sonoma,3,0


### Success Rates

In [86]:
df.sample()

,project_app_id,project_cycle,a1_locode,#,atp_id,awarded,ppno,ppno_1,data_origin,geometry,project_status,solicitation_abv,solicitation,soliciting_agency,project_size,a1_imp_agcy_city,a1_imp_agcy_name,a1_proj_partner_agcy,a1_proj_partner_exists,assembly_district,congressional_district,senate_district,a2_county,a2_ct_dist,a2_info_proj_descr,a2_info_proj_loc,a2_info_proj_name,a2_mop_uza_population,a2_mpo,a2_rtpa,a2_proj_scope_summary,a1_imp_agcy_fed_ma_num,a1_imp_agcy_ma,a1_imp_agcy_state_ma_num,a1_imp_agcy_street,a1_imp_agcy_title,a1_imp_agcy_zip,a1_letter_of_intent,a1_proj_partner_title,a2_assem_dist_a,a2_assem_dist_b,a2_assem_dist_c,a2_congress_dist_a,a2_congress_dist_b,a2_congress_dist_c,a2_past_proj,a2_past_proj_qty,a2_proj_lat,a2_proj_long,a2_project_location_map,a2_senate_dist_a,a2_senate_dist_b,a2_senatedistc,a3_plan_active_trans,a3_plan_active_trans_exists,a3_plan_bicycle,a3_plan_bicycle_exists,a3_plan_ped,a3_plan_ped_exists,a3_plan_srts,a3_plan_srts_exists,a3_proj_type,a3_st_bicycle_applies,a3_st_bicycle_pct,a3_st_num_schools,a3_st_ped_applies,a3_st_ped_pct,a3_st_srts,a3_trail_elig_cost,a3_trail_fed_funding,a3_trail_trans_pct,a3_trails,agency_app_num,app_pk,attch_addtl_attachments,attch_app_sig_page,attch_conditions_photos,attch_conditions_project_map,attch_engineeers_checklist,attch_exhibit22_plan,attch_letters_of_support,attch_link,attch_ni_workplan,attch_project_estimate,completed_pdf_form,main_datetime_stamp,a3_plan_none,a3_plan_other,a3_plan_other_desc,a2_output_outcome,a3_current_plan,b_sig_inter_new_bike_boxes,b_class_1,b_class_2,b_class_3,b_class_4,a4_bike_gap_pct,b_light_intersection,b_mid_block_new_rrfb_signal,b_mid_block_surf_improv,b_bsp_new_bikes,b_bike_new_secured_lockers,b_bike_new_racks,b_bsp_new_station,b_other_bike_improv_1,b_other_bike_improv_qty_1,b_other_bike_improv_2,b_other_bike_improv_qty_2,b_light_rdwy_seg,b_sig_inter_timing_improv,b_un_sig_new_rrfb_signal,b_un_sig_cross_surf_improv,a4_easement_support,m_cls_1_trails_widen_recon_exist,m_cls_1_trails_new__less_than_9,m_cls_1_trails_new_over_9,m_non_cls_trails_new,m_other_trail_imprv_1,m_other_trail_improv_qty_1,m_other_trail_imprv_2,m_other_trail_improv_qty_2,m_non_cls_widen_recon_exist,p_amenities_bench,a4_ped_gap_pct,p_mid_block_cross_new_rrfb_signal,p_light_intersection,p_lighting_rdwy_seg,p_mid_block_cross_surf_improv,p_new_ada_ramp,p_sidewlks_new_barrier_protect,p_sidewlks_new_4_to_8,p_sidewlks_new_over_8,p_other_ped_imprv_1,p_other_ped_qty_1,p_other_ped_imprv_2,p_other_ped_qty_2,p_reconstruct_ramp_to_ada_stand,p_sidewlks_reconstruct_enhance_exist,p_sig_inter_enhance_exist_crosswlk,p_sig_inter_new_crosswlk,p_sig_inter_ped_heads,p_sig_inter_shorten_cross,p_sig_inter_timing_improv,p_amenities_trash_can,p_amenities_shade_tree,p_amenities_shade_tree_type,p_un_sig_inter_new_traff_sig,p_un_sig_inter_new_roundabout,p_un_sig_inter_new_rrfb_sig,p_un_sig_inter_shorten_cross,p_un_sig_inter_cross_surface_improv,p_sidewlks_widen_existing,a4_row_100,a4_row_gov_ease,a4_row_private_ease,v_other_traffic_calming_imprv_1,v_speed_feedback_signs,v_other_traffic_calming_qty_1,v_other_traffic_calming_imprv_2,v_other_traffic_calming_qty_2,v_remove_right_turn_pocket,v_remove_travel_ln,v_sig_inter_new_roundabout,v_sig_inter_timing_improv,v_un_sig_inter_new_traf_sig,v_un_sig_inter_new_roundabout,app_fk,details_datetime_stamp,a4_reg_init,a4_reg_init_pct,a4_com_init,a4_com_init_pct,a4_safe_route,a4_safe_route_pct,a4_fl_mile,a4_fl_mile_pct,a4_emp_based,a4_emp_based_pct,a4_other_ni,a4_other_ni_descr,a4_other_ni_pct,a4_wb_audits,a4_bike_classes,a4_ped_classes,a4_demo_events,a4_com_enc,a4_le_methods,a4_com_meetings,a4_classrooms,a4_school_assem,a4_after_school,a4_bike_rodeos,a4_mock_cities,a4_walk_bus,a4_bike_train,a4_com_challenges,a4_srts_enc,a4_srts_le,a4_srts_training,a4_act_other_1,a4_act_other_1_descr,a4_act_other_2,a4_act_other_2_decr,a4_comm_trad_media,a4_comm_large_media,a4_comm_print,a4_comm_social,a4_comm_web,a4_comm_other,a4_comm_other_descr,a4_comm_language,a4_

In [96]:
(df>>group_by(_.data_origin)>>count(_.a1_imp_agcy_name)>>arrange(_.a1_imp_agcy_name))

,data_origin,a1_imp_agcy_name,n
0,Application,Alameda County,6
1,Application,Alameda County Transportation Commission,1
2,Application,Bay Area Toll Authority,1
3,Application,"Berkeley, City of",2
4,Application,California Department of Transportation,1
5,Application,"Concord, City of",1
6,Application,Contra Costa County,5
45,Funded,Contra Costa County,1
7,Application,Contra Costa Transportation Authority (CCTA),1
8,Application,"Corte Madera, Town of",1


In [95]:
(df>>group_by(_.awarded)>>count(_.a1_imp_agcy_name)>>arrange(_.a1_imp_agcy_name))>>spread("awarded", "n")

,a1_imp_agcy_name,N,Y
0,Alameda County,6,0
1,Alameda County Transportation Commission,1,0
2,Bay Area Toll Authority,1,0
3,"Berkeley, City of",2,0
4,California Department of Transportation,1,0
5,"Concord, City of",1,0
6,Contra Costa County,5,1
7,Contra Costa Transportation Authority (CCTA),1,0
8,"Corte Madera, Town of",1,0
9,"Daly City, City of",1,0


In [89]:
>summarize(success_rate = 

,awarded,a1_imp_agcy_name,n
0,N,Alameda County,6
47,Y,Alameda County,0
1,N,Alameda County Transportation Commission,1
48,Y,Alameda County Transportation Commission,0
2,N,Bay Area Toll Authority,1
...,...,...,...
91,Y,"Tiburon, Town of",0
45,N,"Vacaville, City of",1
92,Y,"Vacaville, City of",0
46,N,"Vallejo, City of",1


## NOTES

* might want to switch around Agency Names
   * ex we have "Danville, City of" and we want "City of Danville"

In [100]:
cities = df>>filter(_.a1_imp_agcy_name.str.contains(","))>>select(_.a1_imp_agcy_name)

In [104]:
cities.sample()

,a1_imp_agcy_name
10,"Oakland, City of"


In [110]:
split = cities["a1_imp_agcy_name"].str.split(", ", 1)

/tmp/ipykernel_2283/3243887913.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.


In [ ]:
cities["split_1"] = splitted[mask].str[1]+'-' +splitted[mask].str[-1]

In [109]:
split

154              [San Mateo,  City of]
443             [San Rafael,  City of]
255              [San Ramon,  City of]
372                [Vallejo,  City of]
341             [Santa Rosa,  City of]
365    [South San Francisco,  City of]
312                [Tiburon,  Town of]
434              [Vacaville,  City of]
367               [San Jose,  City of]
233                [Concord,  City of]
201           [Corte Madera,  Town of]
346              [Daly City,  City of]
109               [Danville,  Town of]
416               [Danville,  Town of]
192                 [Dublin,  City of]
369               [Berkeley,  City of]
212               [Berkeley,  City of]
158         [East Palo Alto,  City of]
322             [Emeryville,  City of]
131                [Oakland,  City of]
100                [Oakland,  City of]
409                [Oakland,  City of]
424                [Oakland,  City of]
2                 [Pacifica,  City of]
118              [San Bruno,  City of]
376             [San Carl